In [72]:
import pandas as pd

df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                   'B': {0: 1, 1: 3, 2: 5},
                   'C': {0: 2, 1: 4, 2: 6}})
df

,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


In [73]:
pd.melt(df, value_vars=['B'])

,variable,value
0,B,1
1,B,3
2,B,5


In [74]:
pd.melt(df, value_vars=['B', 'C'])

,variable,value
0,B,1
1,B,3
2,B,5
3,C,2
4,C,4
5,C,6


In [75]:
pd.melt(df, id_vars=['A'], value_vars=['B'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5


In [76]:
melt_ = pd.melt(df, id_vars=['A'], value_vars=['B', 'C'])
melt_

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5
3,a,C,2
4,b,C,4
5,c,C,6


In [77]:
melt_.pivot('A', 'variable', 'value').reset_index()

variable,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


In [78]:
pd.melt(df, id_vars=['A'], value_vars=['B'],
         var_name='myVarname', value_name='myValname')

,A,myVarname,myValname
0,a,B,1
1,b,B,3
2,c,B,5


In [79]:
df.columns = [list('AAC'), list('DEF')]
df

A     C
   D  E  F
0  a  1  2
1  b  3  4
2  c  5  6

In [80]:
pd.melt(df, id_vars=[('A', 'D')], value_vars=[('C', 'F')])

,"(A, D)",variable_0,variable_1,value
0,a,C,F,2
1,b,C,F,4
2,c,C,F,6


In [81]:
mu_melt = pd.melt(df, id_vars=[('A', 'E')], value_vars=[('A', 'D'), ('C', 'F')]) # 分层索引的melt操作
mu_melt

,"(A, E)",variable_0,variable_1,value
0,1,A,D,a
1,3,A,D,b
2,5,A,D,c
3,1,C,F,2
4,3,C,F,4
5,5,C,F,6


In [82]:
mu_melt.columns

Index([('A', 'E'), 'variable_0', 'variable_1', 'value'], dtype='object')

In [83]:
q = mu_melt[mu_melt['variable_0'].values == 'A']
q = q.drop('variable_0', axis=1)
q.rename(columns={'value':'A'}, inplace=True)
q

,"(A, E)",variable_1,A
0,1,D,a
1,3,D,b
2,5,D,c


In [84]:
s = mu_melt[mu_melt['variable_0'].values == 'C']
s = s.drop('variable_0', axis=1)
s.rename(columns={'value':'C'}, inplace=True)
s

,"(A, E)",variable_1,C
3,1,F,2
4,3,F,4
5,5,F,6


In [85]:
mu_melt.drop(['variable_0', 'value',], axis=1, inplace=True)
mu_melt

,"(A, E)",variable_1
0,1,D
1,3,D
2,5,D
3,1,F
4,3,F
5,5,F


In [86]:
mu_melt = pd.merge(mu_melt, s, how='left', on=[('A', 'E'), 'variable_1'])
mu_melt # DataFrame连接

,"(A, E)",variable_1,C
0,1,D,NaN
1,3,D,NaN
2,5,D,NaN
3,1,F,2
4,3,F,4
5,5,F,6


In [87]:
mu_melt = pd.merge(mu_melt, q, how='left', on=[('A', 'E'), 'variable_1'])
mu_melt

,"(A, E)",variable_1,C,A
0,1,D,NaN,a
1,3,D,NaN,b
2,5,D,NaN,c
3,1,F,2,NaN
4,3,F,4,NaN
5,5,F,6,NaN


In [88]:
mu_melt.pivot(index=('A', 'E'),
              columns='variable_1',
              values=['A', 'C']).dropna(axis=1).reset_index() # 还原成功

KeyError: 'E'